In [ ]:
import numpy as np
import pandas as pd
import glob
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')

df.head()

In [ ]:
def get_slice(r):
    case, day, _, slice_id = r.id.split('_')
    
    fns = glob.glob(f'../input/uw-madison-gi-tract-image-segmentation/train/{case}/{case}_{day}/scans/*.png')
    _, _, h, w, hs, ws = fns[0].split('/')[-1][:-4].split('_')
    
    return f"{len(fns)}_{h}_{w}_{hs}_{ws}"

In [ ]:
df['cd']      = df.id.apply(lambda x: "_".join(x.split('_')[:2]))
df['hw']      = df.progress_apply(get_slice, axis=1)
df['slice']   = df.id.apply(lambda x: int(x.split('_')[-1]))
df['isempty'] = df.segmentation.apply(lambda s: isinstance(s, float))

In [ ]:
df['slice'] = df.id.apply(lambda x: int(x.split('_')[-1]))

In [ ]:
df.hw.value_counts()

In [ ]:
df.isempty.value_counts()

In [ ]:
empty_dict = { 
    s:{
        "large_bowel":set(range(1, n_slices + 1)),
        "small_bowel":set(range(1, n_slices + 1)),
        "stomach":set(range(1, n_slices + 1)),
    } for s, n_slices in zip(df.hw.unique(), [int(k.split('_')[0]) for k in df.hw.unique()])}

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    if not row.isempty and row.slice in empty_dict[row.hw][row['class']]:
        empty_dict[row.hw][row['class']].remove(row.slice)

In [ ]:
for k1 in empty_dict:
    for k2 in empty_dict[k1]:
        empty_dict[k1][k2] = list(empty_dict[k1][k2])

In [ ]:
import json
json.dumps(empty_dict)